In [641]:
import numpy as np
import pandas as pd
import matplotlib.pyplot  as  plt
from sklearn.svm import SVR
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from __future__ import print_function
import torch
from sklearn.preprocessing import MinMaxScaler, StandardScaler
%matplotlib inline
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [642]:
housing=pd.read_csv('Housing.csv')
housing.head(5)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [643]:
housing.mainroad = housing.mainroad.map(dict(yes=1,no=0))
housing.guestroom = housing.guestroom.map(dict(yes=1,no=0))
housing.basement = housing.basement.map(dict(yes=1,no=0))
housing.hotwaterheating = housing.hotwaterheating.map(dict(yes=1,no=0))
housing.airconditioning = housing.airconditioning.map(dict(yes=1,no=0))
housing.prefarea = housing.prefarea.map(dict(yes=1,no=0))

housing.head(5)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished


In [644]:
housing.shape

(545, 13)

In [645]:
num_vars=['price', 'area','bathrooms','bedrooms', 'stories', 'parking']
data_set=housing[num_vars]
data_set.head()

,price,area,bathrooms,bedrooms,stories,parking
0,13300000,7420,2,4,3,2
1,12250000,8960,4,4,4,3
2,12250000,9960,2,3,2,2
3,12215000,7500,2,4,2,3
4,11410000,7420,1,4,2,2


In [646]:

data_set.shape


(545, 6)

In [647]:
scaler = MinMaxScaler()
data_set[num_vars]= scaler.fit_transform(data_set[num_vars])
data_set.head()

C:\Users\nrashvan\AppData\Local\Temp\1\ipykernel_11508\1593678094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set[num_vars]= scaler.fit_transform(data_set[num_vars])


,price,area,bathrooms,bedrooms,stories,parking
0,1.000000,0.396564,0.333333,0.6,0.666667,0.666667
1,0.909091,0.502405,1.000000,0.6,1.000000,1.000000
2,0.909091,0.571134,0.333333,0.4,0.333333,0.666667
3,0.906061,0.402062,0.333333,0.6,0.333333,1.000000
4,0.836364,0.396564,0.000000,0.6,0.333333,0.666667


In [648]:
X = data_set.iloc[:, 1:6].values
X.shape
X

array([[0.39656357, 0.33333333, 0.6       , 0.66666667, 0.66666667],
       [0.5024055 , 1.        , 0.6       , 1.        , 1.        ],
       [0.57113402, 0.33333333, 0.4       , 0.33333333, 0.66666667],
       ...,
       [0.13539519, 0.        , 0.2       , 0.        , 0.        ],
       [0.08659794, 0.        , 0.4       , 0.        , 0.        ],
       [0.15120275, 0.        , 0.4       , 0.33333333, 0.        ]])

In [649]:
Y = data_set.iloc[:, 0].values
Y.shape

(545,)

In [650]:
x=torch.tensor(X)
y=torch.tensor(Y)
x

tensor([[0.3966, 0.3333, 0.6000, 0.6667, 0.6667],
        [0.5024, 1.0000, 0.6000, 1.0000, 1.0000],
        [0.5711, 0.3333, 0.4000, 0.3333, 0.6667],
        ...,
        [0.1354, 0.0000, 0.2000, 0.0000, 0.0000],
        [0.0866, 0.0000, 0.4000, 0.0000, 0.0000],
        [0.1512, 0.0000, 0.4000, 0.3333, 0.0000]], dtype=torch.float64)

In [651]:
x.size()

torch.Size([545, 5])

In [652]:
y.size()

torch.Size([545])

In [653]:
y =y.reshape(-1,1)
y.size()

torch.Size([545, 1])

In [654]:
def model(X5,X4, X3, X2, X1, w5, w4, w3 ,w2, w1, b):
    return w5 * X5 + w4 * X4 + w3 * X3 + w2 * X2 + w1 * X1 + b

In [655]:
def loss_fn(y_p, y):
    squared_diffs=(y_p - y) **2
    return squared_diffs.mean()

In [656]:
n_samples= x.shape[0]
n_val= int(0.2 * n_samples)

shuffled_indices= torch.randperm(n_samples)

train_indices= shuffled_indices[:-n_val]
val_indices= shuffled_indices[-n_val:]

n_samples
n_val


109

In [657]:
train_x= x[train_indices]
train_y=y[train_indices]

val_x= x[val_indices]
val_y= y[val_indices]

train_x_n= 0.1 * train_x
val_x_n= 0.1 * val_x


train_y.size()


torch.Size([436, 1])

In [658]:
train_x_n[:,0]

tensor([0.0135, 0.0162, 0.0261, 0.0443, 0.0092, 0.0093, 0.0093, 0.0091, 0.0173,
        0.0144, 0.0093, 0.0340, 0.0216, 0.0302, 0.0332, 0.0324, 0.0299, 0.0189,
        0.0299, 0.0126, 0.0368, 0.0313, 0.0072, 0.0223, 0.0295, 0.0215, 0.0175,
        0.0105, 0.0181, 0.0442, 0.0216, 0.0328, 0.0423, 0.0137, 0.0265, 0.0378,
        0.0299, 0.0299, 0.0113, 0.0416, 0.0165, 0.0066, 0.0184, 0.0128, 0.0099,
        0.0291, 0.0232, 0.0265, 0.0170, 0.0162, 0.0332, 0.0145, 0.0548, 0.0179,
        0.0165, 0.0251, 0.0323, 0.0384, 0.0145, 0.0129, 0.0354, 0.0140, 0.0035,
        0.0415, 0.0265, 0.0235, 0.0170, 0.0043, 0.0242, 0.0093, 0.0164, 0.1000,
        0.0189, 0.0386, 0.0134, 0.0137, 0.0164, 0.0414, 0.0113, 0.0265, 0.0127,
        0.0122, 0.0058, 0.0196, 0.0122, 0.0081, 0.0240, 0.0356, 0.0398, 0.0196,
        0.0462, 0.0265, 0.0402, 0.0167, 0.0078, 0.0196, 0.0126, 0.0196, 0.0390,
        0.0287, 0.0200, 0.0203, 0.0034, 0.0101, 0.0348, 0.0213, 0.0103, 0.0265,
        0.0072, 0.0093, 0.0261, 0.0454, 

In [659]:
train_y.size()

torch.Size([436, 1])

In [660]:
def training_loop(n_epochs, optimizer, params, train_x, val_x, train_y, val_y):
    for epoch in range (1, n_epochs +1):
        
        train_y_p=model(train_x[:,4],  train_x[:,3], train_x[:,2], train_x[:,1], train_x[:,0],*params)
        train_loss= loss_fn(train_y_p, train_y)
        
        val_y_p=model(val_x[:,4], val_x[:,3], val_x[:,2],val_x[:,1],val_x[:,0],*params)
        val_loss=loss_fn(val_y_p,val_y)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if epoch <=3 or epoch % 500 ==0:
       
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f}, "
                  f" Validation loss {val_loss.item():.4f} ")
            
    return params

In [661]:
import torch.optim as optim
params=torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate=1e-1
optimizer= optim.SGD([params], lr=learning_rate)


training_loop( n_epochs=3000, optimizer=optimizer,params=params, train_x= train_x_n,
              val_x=val_x_n,train_y=train_y, val_y= val_y)

Epoch 1, Training loss 0.0494,  Validation loss 0.0495 
Epoch 2, Training loss 0.0424,  Validation loss 0.0428 
Epoch 3, Training loss 0.0380,  Validation loss 0.0385 
Epoch 500, Training loss 0.0293,  Validation loss 0.0307 
Epoch 1000, Training loss 0.0287,  Validation loss 0.0302 
Epoch 1500, Training loss 0.0282,  Validation loss 0.0297 
Epoch 2000, Training loss 0.0278,  Validation loss 0.0294 
Epoch 2500, Training loss 0.0275,  Validation loss 0.0291 
Epoch 3000, Training loss 0.0272,  Validation loss 0.0288 


tensor([0.4825, 0.4247, 0.7368, 0.6686, 0.7778, 0.1851], requires_grad=True)

In [662]:
import torch.optim as optim
params=torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate=1e-2
optimizer= optim.SGD([params], lr=learning_rate)


training_loop( n_epochs=3000, optimizer=optimizer,params=params, train_x= train_x_n,
              val_x=val_x_n,train_y=train_y, val_y= val_y)

Epoch 1, Training loss 0.0494,  Validation loss 0.0495 
Epoch 2, Training loss 0.0486,  Validation loss 0.0488 
Epoch 3, Training loss 0.0479,  Validation loss 0.0481 
Epoch 500, Training loss 0.0300,  Validation loss 0.0312 
Epoch 1000, Training loss 0.0299,  Validation loss 0.0312 
Epoch 1500, Training loss 0.0298,  Validation loss 0.0311 
Epoch 2000, Training loss 0.0298,  Validation loss 0.0310 
Epoch 2500, Training loss 0.0297,  Validation loss 0.0310 
Epoch 3000, Training loss 0.0296,  Validation loss 0.0309 


tensor([0.9356, 0.9271, 0.9719, 0.9599, 0.9757, 0.1445], requires_grad=True)

In [663]:
import torch.optim as optim
params=torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate=1e-3
optimizer= optim.SGD([params], lr=learning_rate)


training_loop( n_epochs=3000, optimizer=optimizer,params=params, train_x= train_x_n,
              val_x=val_x_n,train_y=train_y, val_y= val_y)

Epoch 1, Training loss 0.0494,  Validation loss 0.0495 
Epoch 2, Training loss 0.0493,  Validation loss 0.0494 
Epoch 3, Training loss 0.0493,  Validation loss 0.0494 
Epoch 500, Training loss 0.0327,  Validation loss 0.0335 
Epoch 1000, Training loss 0.0304,  Validation loss 0.0315 
Epoch 1500, Training loss 0.0301,  Validation loss 0.0313 
Epoch 2000, Training loss 0.0301,  Validation loss 0.0313 
Epoch 2500, Training loss 0.0300,  Validation loss 0.0313 
Epoch 3000, Training loss 0.0300,  Validation loss 0.0313 


tensor([0.9961, 0.9959, 1.0020, 0.9971, 1.0004, 0.1388], requires_grad=True)

In [664]:
import torch.optim as optim
params=torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate=1e-4
optimizer= optim.SGD([params], lr=learning_rate)


training_loop( n_epochs=3000, optimizer=optimizer,params=params, train_x= train_x_n,
              val_x=val_x_n,train_y=train_y, val_y= val_y)

Epoch 1, Training loss 0.0494,  Validation loss 0.0495 
Epoch 2, Training loss 0.0494,  Validation loss 0.0495 
Epoch 3, Training loss 0.0494,  Validation loss 0.0495 
Epoch 500, Training loss 0.0459,  Validation loss 0.0461 
Epoch 1000, Training loss 0.0430,  Validation loss 0.0433 
Epoch 1500, Training loss 0.0407,  Validation loss 0.0411 
Epoch 2000, Training loss 0.0387,  Validation loss 0.0392 
Epoch 2500, Training loss 0.0372,  Validation loss 0.0377 
Epoch 3000, Training loss 0.0359,  Validation loss 0.0365 


tensor([1.0007, 1.0010, 1.0021, 1.0002, 1.0012, 0.0627], requires_grad=True)

In [665]:
import torch.optim as optim
params=torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate=1e-4
optimizer= optim.Adam([params], lr=learning_rate)


training_loop( n_epochs=3000, optimizer=optimizer,params=params, train_x= train_x_n,
              val_x=val_x_n,train_y=train_y, val_y= val_y)

Epoch 1, Training loss 0.0494,  Validation loss 0.0495 
Epoch 2, Training loss 0.0494,  Validation loss 0.0495 
Epoch 3, Training loss 0.0493,  Validation loss 0.0495 
Epoch 500, Training loss 0.0383,  Validation loss 0.0387 
Epoch 1000, Training loss 0.0332,  Validation loss 0.0339 
Epoch 1500, Training loss 0.0311,  Validation loss 0.0321 
Epoch 2000, Training loss 0.0302,  Validation loss 0.0314 
Epoch 2500, Training loss 0.0297,  Validation loss 0.0310 
Epoch 3000, Training loss 0.0293,  Validation loss 0.0307 


tensor([0.8587, 0.8822, 1.0743, 0.7765, 1.0410, 0.1399], requires_grad=True)